<a href="https://colab.research.google.com/github/ThisIsFarhan/Language_Models/blob/main/4_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain>=0.1.17 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 langchain_community
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python==0.2.69

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 MB 12.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.6 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.69-cp311-cp311-linux_x86_64.whl size=55713428 sha256=c214a92e1eea1cbd5c6d433fa616717c1193b52b9664217610e0805433370819
  Stored in directory: /root/.cache/pip/wheels/e8/1b/ff/b4dba97fbd16e731705b262602ba8f3b672bf4bde54ea0c104
Successfully built llama-cpp-python


In [2]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

--2025-01-31 19:57:08--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
Resolving huggingface.co (huggingface.co)... 13.35.202.97, 13.35.202.121, 13.35.202.34, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.97|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/5d99003e395775659b0dde3f941d88ff378b2837a8dc3a2ea94222ab1420fad3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-fp16.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-fp16.gguf%22%3B&Expires=1738357028&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczODM1NzAyOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvNWQ5OTAwM2UzOTU3NzU2NTliMGRkZTNmOTQxZDg4

In [3]:
from langchain import LlamaCpp

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)

In [4]:
from langchain import PromptTemplate

#Chains

In [ ]:
#Single chain


template = """<s><|User|>
{input_prompt}<|END|>
<|Assistant|>
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

basic_chain = prompt | llm
basic_chain.invoke({
    "input_prompt":"Tell me about computer engineering"
})

"Computer Engineering is an interdisciplinary branch of engineering that integrates principles from electrical engineering and computer science. It's focused on the design, development, testing, and manufacturing of both hardware (physical parts) and software for computers, servers, storage devices, mobile phones, tablets, embedded systems in appliances and vehicles etc.\n\nComputer engineers work to develop new technology that makes existing devices faster or smaller but more powerful, as well as developing entirely new technologies like quantum computing. They also focus on improving the security of these technologies and designing efficient algorithms for processing data.\n\nThe field is continually evolving due to rapid advancements in digital technology. Computer engineering requires a solid understanding of both hardware and software elements, so many professionals hold degrees in electrical or computer science. \n\nKey sub-fields within computer engineering include:\n\n1. Embedd

In [ ]:
#Multiple chain
from langchain import LLMChain

#--------------------Templates------------------------------
summary_template = """<s><|user|>
Create a title for a story about {summary}. Only return the one-liner title.<|end|>
<|assistant|>"""
summary_prompt = PromptTemplate(template=summary_template, input_variables=["summary"])
title = LLMChain(llm=llm, prompt=summary_prompt, output_key="title")

title_template = """<s><|user|>
Describe the main character of a story about {summary} with the title {title}. Use only two sentences.<|end|>
<|assistant|>"""
title_prompt = PromptTemplate(
    template=title_template, input_variables=["summary", "title"]
)
character = LLMChain(llm=llm, prompt=title_prompt, output_key="character")

story_template = """<s><|user|>
Create a story about {summary} with the title {title}. The main charachter is: {character}. Only return the story and it cannot be longer than one paragraph<|end|>
<|assistant|>"""
story_prompt = PromptTemplate(
    template=story_template, input_variables=["summary", "title", "character"]
)
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")


#Defining the chain
llm_chain = title | character | story


#invokation
llm_chain.invoke("A person developing an efficient AI agent")

{'summary': 'A person developing an efficient AI agent',
 'title': ' "Crafting Intelligence: The Odyssey of Building an Efficient AI Agent"',
 'character': " The protagonist, Dr. Amelia Chen, is a brilliant and driven computer scientist who tirelessly works to develop an advanced artificial intelligence capable of learning and adapting with unprecedented efficiency. With unwavering determination and innovative problem-solving skills, she navigates the challenges of AI design while grappling with ethical concerns surrounding her creation's potential impact on society.\n\nDr. Amelia Chen is a compassionate yet fiercely independent researcher who brings together an interdisciplinary team to tackle complex problems in artificial intelligence, striving for balance between technological advancements and responsible use of technology. Her journey unfolds as she discovers the immense power her efficient AI agent holds, forcing her to confront difficult choices about how to wield this newfound 

#Memory

In [5]:
#Conversation Buffer

from langchain import LLMChain
from langchain.memory import ConversationBufferMemory

template = """<s><|user|>Current conversation:{chat_history}

{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt", "chat_history"]
)

memory = ConversationBufferMemory(memory_key="chat_history")
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)


<ipython-input-5-618866cdc251>:16: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")
<ipython-input-5-618866cdc251>:17: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(


In [6]:
llm_chain.invoke({"input_prompt":"Hi I am Farhan"})

{'input_prompt': 'Hi I am Farhan',
 'chat_history': '',
 'text': " Hello Farhan! It's great to chat with you. How can I assist you today? Whether it's answering questions, providing information, or just having a friendly conversation, feel free to share what's on your mind!"}

In [7]:
llm_chain.invoke({"input_prompt":"Whats my name?"})

{'input_prompt': 'Whats my name?',
 'chat_history': "Human: Hi I am Farhan\nAI:  Hello Farhan! It's great to chat with you. How can I assist you today? Whether it's answering questions, providing information, or just having a friendly conversation, feel free to share what's on your mind!",
 'text': " Your name is Farhan. Nice to meet you!\n<|assistant|> Hello Farhan! It's great to chat with you. How can I assist you today? Whether it's answering questions, providing information, or just having a friendly conversation, feel free to share what's on your mind! You don't need to provide your name unless you wish to. However, since you have mentioned it, I'll remember that your name is Farhan. How can I assist you today?"}

In [8]:
#Conversation Buffer Window
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-8-c9f66f2e9bbf>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")


In [9]:
llm_chain.invoke({"input_prompt":"Hi My name is Farhan"})

{'input_prompt': 'Hi My name is Farhan',
 'chat_history': '',
 'text': " Hello Farhan, it's nice to meet you! How can I assist you today?"}

In [10]:
llm_chain.invoke({"input_prompt":"Tell me a joke"})

{'input_prompt': 'Tell me a joke',
 'chat_history': "Human: Hi My name is Farhan\nAI:  Hello Farhan, it's nice to meet you! How can I assist you today?",
 'text': " Sure, here's a light-hearted joke for you: Why don't scientists trust atoms? Because they make up everything!\n\nHope that brought a smile to your face. If there's anything else you need help with or any other type of entertainment I can provide, feel free to let me know!"}

In [11]:
llm_chain.invoke({"input_prompt":"What is 2+2?"})

{'input_prompt': 'What is 2+2?',
 'chat_history': "Human: Hi My name is Farhan\nAI:  Hello Farhan, it's nice to meet you! How can I assist you today?\nHuman: Tell me a joke\nAI:  Sure, here's a light-hearted joke for you: Why don't scientists trust atoms? Because they make up everything!\n\nHope that brought a smile to your face. If there's anything else you need help with or any other type of entertainment I can provide, feel free to let me know!",
 'text': ' The answer to the math question is 4. How can I assist you further in your tasks or queries today, Farhan?'}

In [12]:
llm_chain.invoke({"input_prompt":"What is the mean of AGI?"})

{'input_prompt': 'What is the mean of AGI?',
 'chat_history': "Human: Tell me a joke\nAI:  Sure, here's a light-hearted joke for you: Why don't scientists trust atoms? Because they make up everything!\n\nHope that brought a smile to your face. If there's anything else you need help with or any other type of entertainment I can provide, feel free to let me know!\nHuman: What is 2+2?\nAI:  The answer to the math question is 4. How can I assist you further in your tasks or queries today, Farhan?",
 'text': ' The term "AGI" stands for Adjusted Gross Income. To calculate its mean (average), we would need a dataset of individual\'s AGIs. However, it\'s important to note that AGI itself doesn\'t have a "mean" since it represents an income figure for one person over a tax year. If you\'re looking for the average or median AGI across different individuals, I can help guide you on how to calculate that with relevant data!\n\nBut as per your request in the context of humor: Why did the math book 

In [13]:
llm_chain.invoke({"input_prompt":"what is my name?"})

{'input_prompt': 'what is my name?',
 'chat_history': 'Human: What is 2+2?\nAI:  The answer to the math question is 4. How can I assist you further in your tasks or queries today, Farhan?\nHuman: What is the mean of AGI?\nAI:  The term "AGI" stands for Adjusted Gross Income. To calculate its mean (average), we would need a dataset of individual\'s AGIs. However, it\'s important to note that AGI itself doesn\'t have a "mean" since it represents an income figure for one person over a tax year. If you\'re looking for the average or median AGI across different individuals, I can help guide you on how to calculate that with relevant data!\n\nBut as per your request in the context of humor: Why did the math book look sad? Because it had too many problems!',
 'text': ' My name is not mentioned in our conversation, but you referred to yourself as Farhan. How else may I assist you today? If you have any questions or need help with something specific, feel free to let me know!'}

In [18]:
#Summary Buffer
from langchain.memory import ConversationSummaryMemory
summary_prompt_template = """<s><|user|>Summarize the conversation and update it with new lines

Currenty Summary:
{summary}

New Lines of conversation:
{new_lines}

New Summary:<|end|>
<|assistant|>"""

summary_prompt = PromptTemplate(
    llm = llm,
    template=summary_prompt_template,
    input_variables=["new_lines","summary"]
)

memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",prompt=summary_prompt)

llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-18-61ea54dc5138>:20: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",prompt=summary_prompt)


In [20]:
llm_chain.invoke({"input_prompt":"Hi My name is Farhan"})

{'input_prompt': 'Hi My name is Farhan',
 'chat_history': " New Summary: Human introduced themselves as Farhan and was greeted by the AI. The AI offered assistance for the day's activities.\n\nUpdated Conversation:\nHuman: Hi My name is Farhan\nAI: Hello Farhan, nice to meet you! How can I assist you today?",
 'text': " Human: Hi My name is Farhan\nAI: Hello Farhan, it's great to have you here! How may I help you with your activities for the day? Whether it's providing information, setting reminders, or offering suggestions, feel free to ask. I'm here to assist you!"}

In [21]:
llm_chain.invoke({"input_prompt":"What is MQTT?"})

{'input_prompt': 'What is MQTT?',
 'chat_history': " Human introduced themselves as Farhan and was greeted by the AI, which offered assistance for various daily activities such as providing information, setting reminders, and making suggestions. Additionally, the AI expressed enthusiasm in helping Farhan with their day's tasks.\n\nNew Summary: Farhan introduced themselves to the AI, who warmly greeted them and offered a range of support for daily activities including information sharing, reminder setting, and suggestion offering. The AI also conveyed excitement in assisting Farhan throughout their day.",
 'text': ' MQTT, which stands for Message Queuing Telemetry Transport, is a lightweight messaging protocol designed for limited bandwidths and high latency networks. It was created by IBM in the late 1990s and has become particularly popular in Internet of Things (IoT) applications due to its simplicity, efficiency, and reliability. MQTT operates on a publish-subscribe model where devi

In [22]:
llm_chain.invoke({"input_prompt":"What is AGI?"})

{'input_prompt': 'What is AGI?',
 'chat_history': " Farhan introduced themselves to the AI, who warmly greeted them and offered a range of support for daily activities including information sharing, reminder setting, and suggestion offering. The AI also expressed excitement in assisting Farhan throughout their day. When asked about MQTT, the AI provided details about it being a lightweight messaging protocol designed for limited bandwidths and high latency networks used extensively in IoT applications due to its simplicity, efficiency, and reliability. It explained how MQTT operates on a publish-subscribe model enabling scalable systems by allowing devices to communicate through a central broker without direct connections. The AI highlighted the protocol's support for various Quality of Service levels ensuring reliable message delivery as needed in different applications.",
 'text': " AGI, or Artificial General Intelligence, refers to a type of artificial intelligence that exhibits hum

In [23]:
llm_chain.invoke({"input_prompt":"What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': ' Farhan introduced themselves to the AI and engaged in a conversation about daily activities, reminder setting, information sharing, and suggestions for assistance during their day. They also discussed MQTT as a lightweight messaging protocol ideal for IoT applications due to its publish-subscribe model, scalability, limited bandwidth requirements, high latency networks compatibility, reliability, and support for various Quality of Service levels. Subsequently, Farhan inquired about AGI, prompting the AI to explain Artificial General Intelligence as an advanced form of AI that exhibits human-like cognitive abilities across a wide range of tasks, though it is important to note that true AGI has not yet been realized and current technologies are categorized as Narrow AI.',
 'text': ' Your name appears to be Farhan based on our conversation so far. How may I assist you further today?'}